In [24]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import math

In [277]:
def addEdgemap(start, end, edge_x, edge_y, lengthFrac=1, arrowPos = None, arrowLength=0.025, arrowAngle = 30, dotSize=20):
    
    # Get start and end cartesian coordinates
    x0, y0 = start
    x1, y1 = end
    
    # Incorporate the fraction of this segment covered by a dot into total reduction
    length = math.sqrt( (x1-x0)**2 + (y1-y0)**2 )
    dotSizeConversion = .0565/20 # length units per dot size
    convertedDotDiameter = dotSize * dotSizeConversion
    lengthFracReduction = convertedDotDiameter / length
    lengthFrac = lengthFrac - lengthFracReduction

    # Append line corresponding to the edge
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None) # Prevents a line being drawn from end of this edge to start of next edge
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    
    
    if not arrowPos == None:
        
        # Find the point of the arrow; assume is at end unless told middle
        pointx = x1
        pointy = y1
        eta = math.degrees(math.atan((x1-x0)/(y1-y0)))
        
        if arrowPos == 'middle' or arrowPos == 'mid':
            pointx = x0 + (x1-x0)/2
            pointy = y0 + (y1-y0)/2
            
        # Find the directions the arrows are pointing
        signx = (x1-x0)/abs(x1-x0)
        signy = (y1-y0)/abs(y1-y0)
        
        # Append first arrowhead
        dx = arrowLength * math.sin(math.radians(eta + arrowAngle))
        dy = arrowLength * math.cos(math.radians(eta + arrowAngle))    
        edge_x.append(pointx)
        edge_x.append(pointx - signx**2 * signy * dx)
        edge_x.append(None)
        edge_y.append(pointy)
        edge_y.append(pointy - signx**2 * signy * dy)
        edge_y.append(None)
        
#         # And second arrowhead
        dx2 = arrowLength * math.sin(math.radians(eta - arrowAngle))
        dy2 = arrowLength * math.cos(math.radians(eta - arrowAngle))    
        edge_x.append(pointx)
        edge_x.append(pointx - signx**2 * signy * dx2)
#         edge_x.append(None)
        edge_y.append(pointy)
        edge_y.append(pointy - signx**2 * signy * dy2)
#         edge_y.append(None)
        
        edge_x.append(pointx - signx**2 * signy * dx)
        edge_y.append(pointy - signx**2 * signy * dy)
        edge_x.append(None)
        edge_y.append(None)
        

    
    return edge_x, edge_y

In [265]:
towers=pd.read_csv('../data/towers_min.csv',index_col=0)
calls=pd.read_csv('../data/data.csv')

In [266]:
df=pd.merge(calls,towers,on='TowerID')

In [267]:

people=dict(type='scattermapbox',lat=df['lat'],lon=df['lon'],mode='markers')
fig=go.Figure(people,layout={
    'mapbox_style':'open-street-map',
  
    'mapbox':dict(
     
        bearing=0,
        center=dict(
            lat=23.2599,
            lon=77.4126
        ),
        pitch=0,
        zoom=10
    )
    
})

In [268]:
fig.show()

In [269]:
df['Date']=pd.to_datetime(df['Date'],format='%d-%m-%Y')
f=df[df['Date'].isin(pd.date_range('2020/06/05','2020/06/10'))&((df['Caller']==9109709886))].sort_values(['Date','Time'])


In [270]:
f

,Caller,Receiver,Date,Time,Duration,TowerID,IMEI,radio,lon,lat,range
2,9109709886,7694823496,2020-06-06,02:51:03,23,40478-38009-10112,60701793772294,GSM,77.422714,23.184586,1000
88,9109709886,7217467582,2020-06-06,16:13:35,33,40493-1150-38981,910312854667165,GSM,77.400742,23.268013,1000
135,9109709886,7340496772,2020-06-07,01:44:39,36,40458-2131-13202,446206666152640,GSM,77.402802,23.231277,1000
41,9109709886,9217826663,2020-06-10,14:30:19,76,40467-61302-43981,976575300749278,GSM,77.410355,23.196945,1000
119,9109709886,7230137725,2020-06-10,19:55:56,80,40478-38004-10762,430484018436777,GSM,77.435074,23.214798,1000


In [284]:
all_lat=[]
all_lon=[]
locs=[]
for (index,row) in f.iterrows():
    locs.append([row['lat'],row['lon']])
for i in range(len(locs)-1):
    all_lat,all_lon=addEdgemap(locs[i],locs[i+1],all_lat,all_lon, 1, "end",0.009, 30, 10)


In [285]:
print(all_lat)

[23.184585571289002, 23.268013, None, 23.268013, 23.26162187315502, None, 23.268013, 23.25932971179956, 23.26162187315502, None, 23.268013, 23.231277, None, 23.231277, 23.23880705778951, None, 23.231277, 23.23931094822697, 23.23880705778951, None, 23.231277, 23.196945190430004, None, 23.196945190430004, 23.203590567403655, None, 23.196945190430004, 23.20552422970978, 23.203590567403655, None, 23.196945190430004, 23.214797973632997, None, 23.214797973632997, 23.206586469865446, None, 23.214797973632997, 23.213882582921038, 23.206586469865446, None]


In [286]:
edge_trace=go.Scattermapbox(lat=all_lat,lon=all_lon,mode = "lines",)
# edge_trace=go.Scattermapbox(lat=f['lat'],lon=f['lon'],mode = "lines+markers",)
fig=go.Figure(edge_trace,layout={
    'mapbox_style':'open-street-map',
  
    'mapbox':dict(
     
        bearing=0,
        center=dict(
            lat=23.2599,
            lon=77.4126
        ),
        pitch=0,
        zoom=10
    )
    
})
fig.show()